### Cryptocurrency Historical Data Scrapper

1. In this python script we use binance api to get access to www.binance.com
2. The python library package used for scrapping --> https://pypi.org/project/python-binance/
3. www.binance.us is made for use in USA. Change python lib is needed, otherwise script might fail while accessing binance.us. 
4. Edit the library code to change all occurrences of 'www.binance.com' to 'www.binance.us'
5. Symbol_list variable contains list of all ticker symbols of interest
6. This script retains ['open', 'high', 'low', 'close', 'volume'] values of bitcoin and ['close', 'volume'] values of other currencies.
7. This script downloads data for all tickers given in the symbol_list for a given date range.
8. The data is first downloaded as individual files and then combined into single csv file

P.S :- In order to generate the API keys and secrets we refered to the following site :-
https://www.binance.com/en/support/articles/360002502072

In [5]:
pip install python-binance

  Using cached python_binance-0.7.9-py2.py3-none-any.whl (36 kB)
  Using cached dateparser-1.0.0-py2.py3-none-any.whl (279 kB)
  Using cached autobahn-21.3.1-py2.py3-none-any.whl (495 kB)
  Using cached service_identity-18.1.0-py2.py3-none-any.whl (11 kB)
  Using cached Twisted-21.2.0-py3-none-any.whl (3.1 MB)
  Using cached tzlocal-2.1-py2.py3-none-any.whl (16 kB)
  Using cached txaio-21.2.1-py2.py3-none-any.whl (30 kB)
  Using cached hyperlink-21.0.0-py2.py3-none-any.whl (74 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl (7.9 kB)
  Using cached Automat-20.2.0-py2.py3-none-any.whl (31 kB)
  Using cached incremental-21.3.0-py2.py3-none-any.whl (15 kB)
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2

In [226]:
import pandas as pd
import os
import time
from binance.client import Client
from datetime import timedelta, datetime
import pytz
import numpy as np

In [227]:
epoch = 0
dateFormat = "%Y-%m-%d %H:%M:%S"  # e.g. 2021-03-15 20:25:16
columns = ['open','high', 'low', 'close', 'volume', 'close_time', 'quote_av','trades', 'tb_base_av', 
           'tb_quote_av', 'ignore']

columnsofInterest = ['open', 'high', 'low', 'close', 'volume']

In [228]:
def apiConfig():
    # refer: https://www.binance.com/en/support/articles/360002502072 for API keys
    binance_api_key = "plpBiyfaHhJLfVKdl9tZnHhpCpCGJfBxpXE3Adg1sRcYTismr5AehJBkGgGDVUXn"
    binance_api_secret = "KwiTSwHM9acigPaQ7ZpODrznfHHVeoOBVCJ6RVaS7ZVRzdT1QCgDghzQz4bVxUXn"
    binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)
    global epoch
    epoch = datetime.utcfromtimestamp(0) #gives us the date and time in UTC(Coordinated Universal Time)
    return binance_client

In [229]:
def convertTimeToUTC(pstTime):
    #Coordinated Universal Time / Greenwich Mean Time
    utc = pytz.utc
    pst = pytz.timezone('America/Los_Angeles')
    datetime1 = datetime.strptime(pstTime, dateFormat)
    pstTime = pst.localize(datetime1)
    return pstTime.astimezone(utc).strftime(dateFormat)

In [230]:
def convertTimeToPST(utcTime):
    #Phillipine Standard Time 
    datetimeObj = datetime.strptime(utcTime, dateFormat)
    return datetimeObj.replace(tzinfo=time.timezone('UTC')).strftime(fmt)

In [231]:
def toUnixmillis(fromDate):
    #converts timedelta difference to milliseconds
    fromDateObj = datetime.strptime(fromDate, dateFormat)
    past = datetime(1970, 1, 1, tzinfo=fromDateObj.tzinfo)
    return int((fromDateObj - past).total_seconds() * 1000.0)

In [232]:
def toDatetime(ms):
    return datetime.fromtimestamp(int(float(ms) / 1000.0))

In [233]:
def scrapeDataFromBinance(symbol, fromDate, toDate, outputFilename, step=0, pause=-1, simulate=False):
    """
    :param symbol: cryptocurrency symbol specified in the main method
    :param from_date: specified in the main method
    :param to_date: specified in the main method
    :param output_filename: specified in the main method
    :param step: step in number of days. Download data in batches of days given by 'step'
    :param pause: pause seconds before downloading next batch.
        if pause == -1 --> random sleep(2,5)
        if pause == 0 --> no sleep
        if pause == num--> sleep for num of seconds
    :param simulate:
    :return:
    """
    binanceClient = apiConfig()
    
    #creates datetime object from the given string
    fromDateObj = datetime.strptime(fromDate, dateFormat)
    
    #Python timedelta() function is present under datetime library which is generally used for 
    #calculating differences in dates and also can be used for date manipulations in Python.  
    stepDateObj = fromDateObj + timedelta(days=step)
    
    # returns a string representing date and time using date, time or datetime object.
    stepDate = stepDateObj.strftime(dateFormat)

    fromMillis = toUnixmillis(fromDate)
    toMillis = toUnixmillis(toDate)
    stepMillis = toUnixmillis(stepDate)
    #in this case fromMillis equals stepMillis as the step parameter is zero

    count = 0

    while True:
        fromMillisStr = str(fromMillis)
        stepMillisStr = str(stepMillis)
        print('Step %d:Downloading data from %s to %s' % (count,str(toDatetime(fromMillisStr)),
                                                          str(toDatetime(stepMillisStr))
                                                          ))
        if not simulate:
            # download data

            klines = binanceClient.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE,
                                                          fromMillisStr, end_str=stepMillisStr)
            klinesLength = len(klines)
            if klinesLength == 0:
                print('\t Failed to download from %s to %s. Got %d' % (str(toDatetime(fromMillisStr)),
                                                                       str(toDatetime(stepMillisStr)), klinesLength
                                                                       ))
                time.sleep(5)

            print('\t Downloaded data of len %d from %s to %s' % (klinesLength,
                                                                  str(toDatetime(fromMillisStr)),
                                                                  str(toDatetime(stepMillisStr))
                                                                  ))
            newColumns = [item + '_' + symbol for item in columns]
            newColumns.insert(0, 'timestamp')

            df = pd.DataFrame(klines, columns=newColumns)
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            
            # if file does not exist write header 
            if not os.path.isfile('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + outputFilename):
                df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + outputFilename)
            else: # else it exists so append without writing the header
               df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + outputFilename, mode='a', header=False)

        # move to next step of batches
        fromMillis = stepMillis
        stepDateObj = stepDateObj + timedelta(days=step)
        stepDate = stepDateObj.strftime(dateFormat)
        stepMillis = toUnixmillis(stepDate)
        count = count + 1
        if pause == -1:
            pause = np.random.randint(2, 5)
        time.sleep(pause)
        if stepMillis >= toMillis:
            break

In [234]:
dfList = []

In [235]:
def concatenateData(symbolList, outputFilename):
    #dfList = []
    for num, symbol in enumerate(symbolList):
        filename = str('%s_binance_data.csv' % (symbol))
        df = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + filename, index_col=0)
        #df.drop_duplicates(keep='first')
        dfList.append(df)
    
    #df_diff = pd.concat([dfList[0],dfList[1]])

    """
    result = pd.concat(dfList, axis=1, sort=True)
    result.index = pd.to_datetime(df.index)
    result = result.sort_index().drop_duplicates(keep='first')
    idx = np.unique(result.index, return_index=True)[1]
    result = result.iloc[idx]

    newColumns = [item + '_' + 'BTCUSDT' for item in columnsofInterest]
    # new_columns.insert(0, 'timestamp')

    for num, symbol in enumerate(symbolList):
        if symbol == 'BTCUSDT':
            continue
        newColumns.append('close_' + symbol)
        newColumns.append('volume_' + symbol)

    result = result[newColumns]
    result.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + outputFilename)
    """

In [236]:
def removeDuplicatesByIndex(outputFilename):
    result = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + outputFilename, index_col=0)
    result.index = pd.to_datetime(result.index)
    result = result.sort_index()        # drop_duplicates(keep='first')
    idx = np.unique(result.index, return_index=True)[1]
    result = result.iloc[idx]
    result.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + outputFilename)

In [237]:
def processData(masterOutputFilename, concatOutputFilename):
    #master_df = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/' + masterOutputFilename, na_filter=False)
    new_df = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + concatOutputFilename)
    #master_df = master_df.append(new_df)
    new_df.set_index('timestamp', inplace=True)
    new_df.index = pd.to_datetime(new_df.index)
    new_df = new_df.sort_index().drop_duplicates(keep='first')
    new_df.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/' + masterOutputFilename)
    
    

In [241]:
if __name__ == '__main__':
    fromDate = '2018-01-01 00:00:00'
    toDate = time.strftime(dateFormat, time.localtime())
    # UTC time is 8 hrs ahead of PST
    #to_date = '2019-11-19 00:00:00'
    
    #symbols list consists of Litecoin, Ethereum, Bitcoin
    symbolList = ['LTCUSDT', 'ETHUSDT', 'BTCUSDT']
    
    
    for num, symbol in enumerate(symbolList):
        outputFilename = '%s_binance_data.csv' % (symbol)
        print('-' * 60)
        print('Downloading data from %s to %s for %s' % (fromDate, toDate, symbol))
        print('-' * 60)
        scrapeDataFromBinance(symbol, fromDate, toDate, outputFilename, step=1, pause=-1, simulate=False)
    

    # concatenate all the cryptocurrency data into a single csv file
    concatOutputFilename = 'crypto_data_final_cleaned.csv'
    concatenateData(symbolList, concatOutputFilename)

    # Append the results to master cryptocurrency csv data 
    masterOutputFilename = 'crypto_data_master_cleaned.csv'
    processData(masterOutputFilename, concatOutputFilename)
    removeDuplicatesByIndex(masterOutputFilename)

------------------------------------------------------------
------------------------------------------------------------
Step 0:Downloading data from 2018-01-01 00:00:00 to 2018-01-02 00:00:00
	 Downloaded data of len 1441 from 2018-01-01 00:00:00 to 2018-01-02 00:00:00
Step 1:Downloading data from 2018-01-02 00:00:00 to 2018-01-03 00:00:00
	 Downloaded data of len 1441 from 2018-01-02 00:00:00 to 2018-01-03 00:00:00
Step 2:Downloading data from 2018-01-03 00:00:00 to 2018-01-04 00:00:00
	 Downloaded data of len 1441 from 2018-01-03 00:00:00 to 2018-01-04 00:00:00
Step 3:Downloading data from 2018-01-04 00:00:00 to 2018-01-05 00:00:00
	 Downloaded data of len 1316 from 2018-01-04 00:00:00 to 2018-01-05 00:00:00
Step 4:Downloading data from 2018-01-05 00:00:00 to 2018-01-06 00:00:00
	 Downloaded data of len 1441 from 2018-01-05 00:00:00 to 2018-01-06 00:00:00
Step 5:Downloading data from 2018-01-06 00:00:00 to 2018-01-07 00:00:00
	 Downloaded data of len 1441 from 2018-01-06 00:00:00 t

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/crypto_data_final_cleaned.csv'

In [225]:
type(dfList[2])

pandas.core.frame.DataFrame

In [242]:
df_bitcoin = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/BTCUSDT_binance_data.csv')
df_ethereum = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/ETHUSDT_binance_data.csv')
df_litecoin = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Historical Data/LTCUSDT_binance_data.csv')

In [243]:
df_bitcoin.shape

(1684570, 12)

In [244]:
df_ethereum.shape

(1684569, 12)

In [245]:
df_litecoin.shape

(1684568, 12)

In [246]:
df_bitcoin.tail(5)

,timestamp,open_BTCUSDT,high_BTCUSDT,low_BTCUSDT,close_BTCUSDT,volume_BTCUSDT,close_time_BTCUSDT,quote_av_BTCUSDT,trades_BTCUSDT,tb_base_av_BTCUSDT,tb_quote_av_BTCUSDT,ignore_BTCUSDT
1684565,2021-03-19 23:56:00,58017.60,58098.21,58005.82,58072.98,27.475946,1616198219999,1.595135e+06,862,16.978483,9.856744e+05,0
1684566,2021-03-19 23:57:00,58081.46,58183.15,58065.00,58074.35,25.483689,1616198279999,1.481252e+06,988,13.714531,7.971212e+05,0
1684567,2021-03-19 23:58:00,58065.00,58129.96,58060.75,58122.74,16.586220,1616198339999,9.635101e+05,553,8.015072,4.655924e+05,0
1684568,2021-03-19 23:59:00,58122.73,58129.96,58030.01,58030.01,49.865671,1616198399999,2.897068e+06,950,18.574515,1.078977e+06,0
1684569,2021-03-20 00:00:00,58030.01,58059.20,57958.00,58005.29,93.114043,1616198459999,5.401450e+06,3480,32.915077,1.909478e+06,0


In [247]:
df_litecoin.tail(5)

,timestamp,open_LTCUSDT,high_LTCUSDT,low_LTCUSDT,close_LTCUSDT,volume_LTCUSDT,close_time_LTCUSDT,quote_av_LTCUSDT,trades_LTCUSDT,tb_base_av_LTCUSDT,tb_quote_av_LTCUSDT,ignore_LTCUSDT
1684563,2021-03-19 23:56:00,199.90,200.00,199.78,199.97,218.54908,1616198219999,43695.752627,99,92.39820,18477.015156,0
1684564,2021-03-19 23:57:00,199.99,200.31,199.99,200.15,447.09300,1616198279999,89452.517844,117,310.95121,62203.210496,0
1684565,2021-03-19 23:58:00,200.13,200.25,200.09,200.18,113.39914,1616198339999,22701.457523,64,57.70894,11553.039748,0
1684566,2021-03-19 23:59:00,200.18,200.21,199.95,199.96,592.98543,1616198399999,118638.961153,174,431.44640,86320.041867,0
1684567,2021-03-20 00:00:00,199.96,200.19,199.84,199.91,802.56985,1616198459999,160511.575673,254,572.41216,114500.017067,0


In [248]:
df_ethereum.tail(5)

,timestamp,open_ETHUSDT,high_ETHUSDT,low_ETHUSDT,close_ETHUSDT,volume_ETHUSDT,close_time_ETHUSDT,quote_av_ETHUSDT,trades_ETHUSDT,tb_base_av_ETHUSDT,tb_quote_av_ETHUSDT,ignore_ETHUSDT
1684564,2021-03-19 23:56:00,1806.45,1809.51,1806.28,1809.17,301.02744,1616198219999,5.441972e+05,325,164.96141,298241.978331,0
1684565,2021-03-19 23:57:00,1809.04,1811.35,1809.02,1809.80,257.41115,1616198279999,4.659404e+05,334,62.24151,112661.722459,0
1684566,2021-03-19 23:58:00,1809.80,1810.91,1809.53,1810.49,257.30829,1616198339999,4.657823e+05,271,91.27548,165237.933762,0
1684567,2021-03-19 23:59:00,1810.49,1810.67,1809.01,1809.10,465.63840,1616198399999,8.427616e+05,431,165.59377,299667.231174,0
1684568,2021-03-20 00:00:00,1809.01,1812.20,1808.25,1809.15,803.72006,1616198459999,1.454433e+06,1200,387.57472,701505.864903,0
